In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np 
from tensorflow.keras import layers

import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.14.0


In [71]:

cancer_type_columns = ["Neoplasm Disease Stage American Joint Committee on Cancer Code", "Neoplasm American Joint Committee on Cancer Clinical Distant Metastasis M Stage", "American Joint Committee on Cancer Metastasis Stage Code", "American Joint Committee on Cancer Lymph Node Stage Code.1", "American Joint Committee on Cancer Lymph Node Stage Code", "Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code", "Neoplasm Disease Stage American Joint Committee on Cancer Code.1", "American Joint Committee on Cancer Publication Version Type", "American Joint Committee on Cancer Tumor Stage Code"]

pd_df_original = pd.read_csv(
    "./Testicular Cancer Dataset.csv",
    # names=[*cancer_type_columns, "Disease Free (Months)"]
    usecols=[*cancer_type_columns, 'Disease Free (Months)', 'Disease Free Status']
)

pd_df_original = pd_df_original.dropna()

pd_df = pd_df_original.copy()

target = pd_df.apply(lambda p: p['Disease Free (Months)'] * (1 if p['Disease Free Status'] == '1:Recurred/Progressed' else 1), axis=1)

pd_df.pop('Disease Free (Months)')
pd_df.pop('Disease Free Status')

inputs = {}

for name, column in pd_df.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

# Vi har ingen numeric inputs pr no

# numeric_inputs = {name:input for name,input in inputs.items()
#                   if input.dtype==tf.float32}

# x = layers.Concatenate()(list(numeric_inputs.values()))
# norm = layers.Normalization()
# norm.adapt(np.array(pd_df_original[numeric_inputs.keys()]))
# all_numeric_inputs = norm(x)

# all_numeric_inputs

preprocessed_inputs = [] # [all_numeric_inputs]

for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  lookup = layers.StringLookup(vocabulary=np.unique(pd_df[name]))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

# preprocessed_inputs

preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

# Nå visualiserings greie
# tf.keras.utils.plot_model(model = titanic_preprocessing , rankdir="LR", dpi=72, show_shapes=True)


my_features_dict = {name: np.array(value) 
                         for name, value in pd_df.items()}

features_dict = {name:values[:1] for name, values in my_features_dict.items()}
preprocessing(my_features_dict)

# Must split my_features_dict into training and testing data
# Split on the same indexes with target, my_features_dict[0] corresponds to target[0] and so on

def model_func(preprocessing, inputs):
    body = tf.keras.Sequential([
        layers.Dense(16),
        layers.Dense(1)
    ])

    preprocessed_inputs = preprocessing(inputs)
    result = body(preprocessed_inputs)
    model = tf.keras.Model(inputs, result)

    model.compile(
        loss=tf.keras.losses.MeanAbsolutePercentageError(),
        optimizer=tf.keras.optimizers.legacy.Adam()
    )
    return model

model = model_func(preprocessing, inputs)

In [75]:
model.fit(x=my_features_dict, y=target, epochs=1000)

Epoch 1/1000
4/4 [==============================] - 0s 1ms/step - loss: 58.1689
Epoch 2/1000
4/4 [==============================] - 0s 2ms/step - loss: 58.5878
Epoch 3/1000
4/4 [==============================] - 0s 2ms/step - loss: 58.1190
Epoch 4/1000
4/4 [==============================] - 0s 2ms/step - loss: 58.4291
Epoch 5/1000
4/4 [==============================] - 0s 1ms/step - loss: 58.6821
Epoch 6/1000
4/4 [==============================] - 0s 2ms/step - loss: 58.6879
Epoch 7/1000
4/4 [==============================] - 0s 2ms/step - loss: 58.8804
Epoch 8/1000
4/4 [==============================] - 0s 1ms/step - loss: 59.6362
Epoch 9/1000
4/4 [==============================] - 0s 2ms/step - loss: 58.4397
Epoch 10/1000
4/4 [==============================] - 0s 1ms/step - loss: 59.1131
Epoch 11/1000
4/4 [==============================] - 0s 2ms/step - loss: 58.2669
Epoch 12/1000
4/4 [==============================] - 0s 2ms/step - loss: 59.5629
Epoch 13/1000
4/4 [==================

In [15]:
model = tf.keras.models.Sequential([
#   tf.keras.layers.Normalization().adapt(csv_data),
  tf.keras.layers.Dense(62),
  tf.keras.layers.Dense(5, activation='relu'),
  tf.keras.layers.Dense(5, activation='relu'),
  tf.keras.layers.Dense(1)
])

loss_fn = tf.keras.losses.MeanAbsolutePercentageError() # Denne brukes for regression

model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.005),
    loss=loss_fn,
    # Kommentere ut metrics=accuracy for å få det til å vær regression (predict en kontinuerlig verdi)
    # metrics=['accuracy'] 
)

# predictions = model(csv_data[:1]).numpy()
# predictions

# loss_fn(csv_result[:1], predictions).numpy()

